# Test avec un MLM

In [6]:
# Test avec un MLM
from transformers import DataCollatorForLanguageModeling, AutoTokenizer

# Charger un tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Créer le data collator pour le Masked Language Modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,  # Activer le masquage de mots
    mlm_probability=0.15  # Probabilité de masquer un jeton
)

# Exemple de phrases tokenisées
tokenized_sentences = [
    {"input_ids": [101, 7592, 1010, 2023, 2003, 102]},
    {"input_ids": [101, 2023, 2003, 1037, 2503, 102]}
]

# Appliquer le data collator
batch = data_collator(tokenized_sentences)

# Le 'batch' contiendra les 'input_ids' avec certains jetons masqués
# et les 'labels' correspondants pour l'entraînement.
print(batch)


{'input_ids': tensor([[ 101,  103, 1010, 2023, 2003,  102],
        [ 101, 2023, 2003, 1037,  103,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1]]), 'labels': tensor([[-100, 7592, -100, -100, -100, -100],
        [-100, -100, -100, -100, 2503, -100]])}


Dans les MLM on utilise un masquage d'un ou plusieurs tokens comme label, c'est représenté par les labels

définition : label = bonne réponse dans une entrée

Exemple de sortie du code ci dessus avec 1 mask
```
{
'input_ids': tensor([
    [ 101, 7592, 1010, 2023, 2003,  102],
    [ 101, 2023,  103, 1037, 2503,  102]
]),
'attention_mask': tensor([
    [1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1]
]),
'labels': tensor([
    [-100, -100, -100, -100, -100, -100],
    [-100, -100, 2003, -100, -100, -100]
])
}
```

Exemple avec 2 masks
```
{
'input_ids': tensor([
    [ 101,  103, 1010, 2023, 2003,  102],
    [ 101, 2023, 2003, 1037,  103,  102]
]),
'attention_mask': tensor([
    [1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1]
]),
'labels': tensor([
    [-100, 7592, -100, -100, -100, -100],
    [-100, -100, -100, -100, 2503, -100]
])
}
```


principe général :

| séquence      | 0    | 1    | 2    | 3      | 4        | 5 |
|---------------|------|------|------|--------|----------|---|
| **entrée**    | La   | chat | a    | quatre | papattes | ! |
| **input_ids** | La   | chat | a    | 103    | papattes | ! |
| **labels**    | -100 | -100 | -100 | quatre | -100     | ! |

Ici le 103 dans input_ids correspond au token MASK, c'est a dire la valeur a masquer.

Le -100 dans label représente que la valeur ne va pas être testée.

Ce qu'il se passe c'est que le model essais de prédire le mot masque et calcul la différence entre la prediction et la bonne réponse pour déterminé la grandeur de l'erreur.


# Test avec le GML

In [9]:
from transformers import AutoTokenizer, DataCollatorForLanguageModeling

# 1. Charger le tokenizer spécifique au modèle CodeGen
#    Ce tokenizer est optimisé pour du code Python.
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")

# Si le tokenizer n'a pas de jeton de padding, on lui en assigne un.
# Le jeton EOS (End Of Sentence) est souvent un bon candidat.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 2. Créer le Data Collator pour le Causal Language Modeling
#    La seule différence est mlm=False.
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # 👈 C'est le changement crucial !
)

# 3. Exemple avec des extraits de code Python
python_code_snippets = [
    "def hello_world():",
    "import numpy as np"
]

# Tokeniser les extraits de code
tokenized_code = tokenizer(python_code_snippets, truncation=True)

# 4. Appliquer le data collator pour créer un lot
batch = data_collator([{"input_ids": item} for item in tokenized_code["input_ids"]])

# 5. Analyser le résultat
print("--- Input IDs ---")
print(batch['input_ids'])

print("\n--- Labels ---")
print(batch['labels'])

batch

--- Input IDs ---
tensor([[ 4299, 23748,    62,  6894, 33529],
        [11748,   299, 32152,   355, 45941]])

--- Labels ---
tensor([[ 4299, 23748,    62,  6894, 33529],
        [11748,   299, 32152,   355, 45941]])


{'input_ids': tensor([[ 4299, 23748,    62,  6894, 33529],
        [11748,   299, 32152,   355, 45941]]), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]]), 'labels': tensor([[ 4299, 23748,    62,  6894, 33529],
        [11748,   299, 32152,   355, 45941]])}

Le principe est fondamentalement différent, le CML va pouvoir prédire le dernier mot de la séquence proposer en entré.

Dans la sequence [ 4299, 23748, 62, 6894, 33529] le model cherchera à deviner 33529 à partir des token [ 4299, 23748, 62, 6894]

le masque d'attention est interne au model et se nome masque d'attention causal


# TODO

* [ ] Approfondir le masque d'attention causal, entre autre comment se déroule la prédition du token suivant la séquence actuellement active.
* [ ] Role du padding mask dans le CML, c'est pas complétement clair.
* [ ] Vérifier le mécanisme de cross attention.

# Sources

## Différence entre MLM et CLM

https://medium.com/data-science/understanding-masked-language-models-mlm-and-causal-language-models-clm-in-nlp-194c15f56a5

https://www.youtube.com/watch?v=agRyKPhsqJ4

## Cross-attention

https://datascience.stackexchange.com/questions/126187/cross-attention-mask-in-transformers

https://arxiv.org/abs/1706.03762
